In [63]:
!pip install CoolProp

In [64]:
from CoolProp.CoolProp import PropsSI

A vapor compression refrigeration cycle with refrigerant-134a as the working fluid is used to maintain a space at - 13 C by rejecting heat to ambient air at 27 C. 

R-134a enters the compressor at 100 kPa superheated by 6.4 C at a rate of 0.05 kg/s:

In [65]:
fluid = 'R134a'

P1 = 100*1e3
Q1 = 1
T1superheat = 6.4
mdot_kgPerS = 0.05

The isentropic efficency of the compressor is 85\%. 

In [66]:
eta = 0.85

The refrigerant leaves the condenser at 39.4 C as a saturated liquid.

In [67]:
T3_C = 39.4
T3 = T3_C + 273.15 

Determine:
- (a) the rate of cooling provided and the COP of the system.
- (b) the exergy destruction in each basic component.
- (c) the minimum power input and the second-law efficiency of the cycle.
- (d) the rate of total exergy destruction. 

All units will be SI.

State 1:

In [68]:
TsatP1 = PropsSI('T','P',P1,'Q',Q1,fluid)

T1 = TsatP1 + T1superheat
T1_C = T1 - 273.15
print('T1 = {0:1.2f} C.'.format(T1_C))

T1 = -19.96 C.


In [69]:
h1 = PropsSI('H','P',P1,'T',T1,fluid)
# print('h1 = {0:1.2f} J/kg.'.format(h1))
s1 = PropsSI('S','P',P1,'T',T1,fluid)

In [70]:
# state 3
Q3 = 0
P3 = PropsSI('P','T',T3,'Q',Q3,fluid)
s3 = PropsSI('S','T',T3,'Q',Q3,fluid)
h3 = PropsSI('H','T',T3,'Q',Q3,fluid)
print('P3 = {0:1.2f} Pa.'.format(P3))

P3 = 1000333.12 Pa.


In [71]:
# state 2
P2 = P3 # isobaric
s2s = s1
h2s = PropsSI('H','P',P2,'S',s2s,fluid)

In [72]:
# state 4
h4 = h3
P4 = P1
s4 = PropsSI('S','P',P4,'H',h4,fluid)

Isentropic efficiency is by definition:
$$\eta_C = \frac{h_{2s} - h_1}{h_2 - h_1} $$

Re-arranging:

In [73]:
h2 = ((h2s - h1)/eta)+h1

In [74]:
s2 = PropsSI('S','P',P2,'H',h2,fluid)

### Part (a)

The refrigeration load, the rate of heat rejected, and the power input are:

In [75]:
mdot = mdot_kgPerS

QdotL = mdot*(h1 - h4)
QdotH = mdot*(h2 - h3)
Wdotin = mdot*(h2 - h1)

print('QL = {0:1.2f} W.'.format(QdotL))
print('QH = {0:1.2f} W.'.format(QdotH))
print('Win = {0:1.2f} W.'.format(Wdotin))

QL = 6608.27 W.
QH = 9527.22 W.
Win = 2918.96 W.


The COP of the refrigeration cycle becomes:

$$\text{COP}_R = \frac{\dot{Q}_L}{\dot{W}_\text{in}} $$

In [76]:
COPr = QdotL/Wdotin
print(COPr)

2.263913759378019


### Part (b)

The dead state temperature is $T_0$, which is the temperature of the environment:

In [77]:
T0 = 27 + 273.15

With this we can determine the exergy destruction in each component of the cycle:
- Compressor, 1-2
- Condenser, 2-3
- Expansion valve, 3-4
- Evaporator, 4-1

$$\dot{X}_\text{dest,1-2} = T_0 \dot{S}_\text{gen,1-2} = T_0\dot{m}\left(s_2 - s_1\right) $$

In [78]:
X12 = T0*mdot*(s2-s1)
print('X12 = {0:1.2f} W.'.format(X12))

X12 = 394.31 W.


$$\dot{X}_\text{dest,2-3} = T_0 \dot{S}_\text{gen,2-3} = T_0 \left[ \dot{m}\left(s_3 - s_2\right) + \frac{\dot{Q}_H}{T_H} \right]$$

In [79]:
TH = T0
X23 = T0*(mdot*(s3-s2)+(QdotH/TH))
print('X23 = {0:1.2f} W.'.format(X23))

X23 = 426.23 W.


$$\dot{X}_\text{dest,3-4} = T_0 \dot{S}_\text{gen,3-4} = T_0\dot{m}\left(s_4 - s_3\right) $$

In [80]:
X34 = T0*mdot*(s4-s3)
print('X34 = {0:1.2f} W.'.format(X34))

X34 = 673.50 W.


$$\dot{X}_\text{dest,4-1} = T_0 \dot{S}_\text{gen,4-1} = T_0 \left[ \dot{m}\left(s_1 - s_4\right) - \frac{\dot{Q}_L}{T_L} \right]$$

We are told that we are trying to keep a space at - 13 C. Therefore, $T_L = - 13 ^{\circ}$C.

In [81]:
TL_C = - 13
TL = TL_C+273.15 
X41 = T0*(mdot*(s1-s4)-(QdotL/TL))
print('X41 = {0:1.2f} W.'.format(X41))

X41 = 408.84 W.


### Part (c)

The exergy flow associated with heat transferred from the low-temperature medium is:

$$\dot{X}_{Q_L} = \dot{Q}_L\frac{T_0 - T_L}{T_L} $$

In [82]:
XQL = QdotL*((T0 - TL)/TL)
print('XQL = {0:1.2f} W.'.format(XQL))

XQL = 1016.07 W.


This is also the minimum or reversible power input for the cycle:
$$\dot{W}_\text{min,in} = \dot{X}_{Q_L} $$

In [83]:
WdotMinIn = XQL

$$\eta_\text{II} = \frac{\dot{X}_{Q_L}}{\dot{W}_\text{in}} $$

In [84]:
eta_II = XQL/Wdotin
print('The second law efficiency of this cycle is {0:1.2f}.'.format(eta_II))

The second law efficiency of this cycle is 0.35.


### Part (d)

The total exergy destruction is the difference between the exergy expended (power input) and the exergy recovered (the exergy of the heat transferred from the low-temperature medium):

$$\dot{X}_\text{dest,total} = \dot{W}_\text{in} - \dot{X}_{\dot{Q}_L}$$

In [85]:
Xtotal = Wdotin - XQL
print('Xtotal = {0:1.2f} W.'.format(Xtotal))

Xtotal = 1902.89 W.


The total exergy destruction can also be calculated by summing the above terms:

In [86]:
Xdesttotal = X12+X23+X34+X41
print('Xdesttotal = {0:1.2f} W.'.format(Xdesttotal))

Xdesttotal = 1902.89 W.


So what's the most irreversible process?

In [87]:
print(X12)
print(X23)
print(X34)
print(X41)

394.3123497098866
426.23192777886044
673.4987349192629
408.8432533768627


It's process 3 to 4, which is the expansion valve. Could we replace this with a turbine?

We can also increase the evaporation temperature and decrease the condensing temperature. 